# **Tutorial #1:** APIを用いたeLabFTWのノート作成

<b>Note:</b> ご不明な点がございましたら、lee.jemyung@naist.ac.jp にご連絡ください

## **Step 1:** APIを使って簡単なデータを入れてみましょう
#### テンプレートを読込んで実験ノートを作成し、簡単なデータを追加する基本的なAPIの使用例

## 前準備
### 「.env」 ファイルの作成
実行ファイル（「\*.py」や「\*.ipynb」）と同じフォルダに以下の内容を入力して「.env」ファイルを保存する。
<br><span style="background-color:red"><b>注</b></span>： "<b>.env</b>" というファイル名で保存してくださ。
<br><br>
「.env」ファイルの内容<br>
    
> <span style="font-size:80%"> HOST_URL='https\://makimono.elab.one/api/v2'   <span style="color: #808080">　<- この部分は変更しません</span></span><br>
> <span style="font-size:80%"> API_KEY='' <span style="color: #808080">　<-「''」の中にコピーしたAPI-KEYをペーストしてください</span></span>

<span style="background-color:blue"><b>参考</b></span>：API-KEYは「Settings」→「API KEYS」→「Create new key」→「Generate an API Key」から取得できます。


### コード駆動準備
ライブラリ、パッケージ、モジュールの呼び出しと設定

In [1]:
import os
import elabapi_python
from elabapi_python.rest import ApiException
import json
import pandas as pd
from IPython.display import display, HTML
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import warnings
from urllib3.exceptions import InsecureRequestWarning
from difflib import get_close_matches

# APIクライアントの設定
load_dotenv()
configuration = elabapi_python.Configuration()
configuration.host = os.environ["HOST_URL"]
configuration.api_key['Authorization'] = os.environ["API_KEY"]
configuration.debug = False
configuration.verify_ssl = False

# SSL警告を無効化
warnings.simplefilter('ignore', InsecureRequestWarning)

# 各種のインスタンスを作成
api_client = elabapi_python.ApiClient(configuration)
api_client.set_default_header(header_name = 'Authorization', header_value = configuration.api_key['Authorization'] )
experimentsApi = elabapi_python.ExperimentsApi(api_client)
templatesApi = elabapi_python.ExperimentsTemplatesApi(api_client)
uploadsApi = elabapi_python.UploadsApi(api_client)
itemsApi = elabapi_python.ItemsApi(api_client)

## 実験ノート作成

### テンプレートリストの確認
現在登録されているテンプレートのリストを表示します。テンプレートのIDとTitleを確認することができます。

In [2]:
# テンプレートIDとタイトルを取得
templates_data = []
errors = []

try:
    templates_list = templatesApi.read_experiments_templates()
    
    for template in templates_list:
        try:
            templates_data.append({
                'ID': template.id,
                'Title': template.title
            })
        except AttributeError as e:
            errors.append(f"AttributeError for template ID {template.id}: {e}")
        except Exception as e:
            errors.append(f"An error occurred for template ID {template.id}: {e}")

except ApiException as e:
    print(f"Exception when calling TemplatesApi->read_experiments_templates: {e}\n")
    if e.body:
        print(f"Error details: {e.body}\n")

# データフレームに変換して表示
templates_df = pd.DataFrame(templates_data)
templates_df[['ID']] = templates_df[['ID']].astype(str)
display(templates_df)

# エラーの内容を表示
if errors:
    print("Errors encountered:")
    for error in errors:
        print(error)

,ID,Title
0,109,Test_No.1
1,108,Test_No.2


### テンプレート指定
上記のテンプレートリストを見て、使用するテンプレートのIDまたはTitleを以下に記入してください。

In [3]:
template_id = ""
template_title = "Test_No1"
sample_id = ""

### テンプレートの割り当て
指定したIDまたはTitleからテンプレートを割り当て、IDとTitleを確定</br>
コード実行時に割り当てられたIDとTitleが表示される。

In [5]:
detected_title = ""
try:
    if template_id and (templates_df['ID'] == template_id).any():
        detected_title = templates_df.loc[templates_df['ID'] == template_id, 'Title'].values[0]
        if not template_title:
            template_title = detected_title
    elif template_title:
        if (templates_df['Title'] == template_title).any():
            detected_title = template_title
        else:
            detected_title = get_close_matches(template_title, templates_df['Title'] , 3, 0.7)
            if detected_title:
                detected_title = detected_title[0]
            else:
                raise ValueError(f"Template title is not detected.")
        template_id = templates_df.loc[templates_df['Title'] == detected_title, 'ID'].values[0]
    else:
        raise ValueError("Please enter a valid template ID or Title, above.")
    
    print(f"Template title is assigned to \"{detected_title}\"" + ("." if detected_title == template_title else f", altered from \"{template_title}\"."))

    if template_id in templates_df['ID'].values:
        print(f"Template ID: {template_id}")
    else:
        raise ValueError(f"Template ID {template_id} not found in templates_df")
except ValueError as e:
    print("ValueError: %s\n" % e)

Template title is assigned to "Test_No.1", altered from "Test_No1".
Template ID: 109


### テンプレートの取得
割り当てられたIDのテンプレートを取得

In [6]:
# Prepare the body for creating an experiment
template = None
experiment_body = {
    "category_id": template_id,  # カテゴリIDを指定
}

try:
    if template_id in templates_df['ID'].values:
        template = templatesApi.get_experiment_template(template_id)
        print(f"Template (ID: {template_id}) is ready to use.")
    else:
        raise ValueError(f"Template ID {template_id} not found in templates dataframe.")
except ValueError as e:
    print("ValueError: %s\n" % e)

Template (ID: 109) is ready to use.


### テンプレートの確認
取得したテンプレートを確認

In [7]:
try:    
    template_dict = template.to_dict()
    template_html = "<h2>Template Information</h2>"
    template_html += "<ul>"
    for key, value in template_dict.items():
        template_html += f"<li><strong>{key}:</strong> {value}</li>"
    template_html += "</ul>"
    display(HTML(template_html))
except ApiException as e:
    print(f"Exception when calling TemplatesApi->get_experiment_template: {e}\n")
    if e.body:
           print(f"Error details: {e.body}\n")
except Exception as e:
    print(f"An error occurred: {e}\n")

### 実験ノートの準備
テンプレートから「(Template title)_(Sample ID)」をタイトルにする実験ノートを作成する。</br>
正常に作成できていたら、その結果を出力し確認する。

In [8]:
try:
    # テンプレートIDがデータフレームに存在するか確認
    if template_id not in templates_df['ID'].values:
        display(templates_df)
        print(template_id, " ", type(template_id))
        raise ValueError(f"Template ID {template_id} not found in templates_df")

    # テンプレートIDに対応するタイトルを取得
    tmpl_title = templates_df.loc[templates_df['ID'] == template_id, 'Title'].values[0]
    
    # 新しいタイトルを作成
    new_title = f"{tmpl_title}_{sample_id}"
    
    # 実験ノートの内容を設定
    experiment_body = {
        "category_id": str(template_id)  # カテゴリIDを指定
    }
   
    # Create an experiment
    api_response = experimentsApi.post_experiment(body=experiment_body)    
    print("Experiment created successfully: ")
    if api_response: print(f"No issue.")
    else: print(api_response)

    # 新しい実験ノートのIDを取得するため、最新の実験ノートを取得
    experiments_list = sorted(experimentsApi.read_experiments(), key=lambda x: x._date, reverse=True)
    new_experiment_id = experiments_list[0].id

    # 実験ノートのタイトルを更新
    update_body = {
        "title": new_title
    }
    update_response = experimentsApi.patch_experiment(new_experiment_id, body=update_body)
    print("Experiment title updated successfully:")
    print(update_response)

except ValueError as e:
    print("ValueError: %s\n" % e)
except ApiException as e:
    print("Exception when calling ExperimentsApi->post_experiment or patch_experiment: %s\n" % e)
    if e.body:
        print(f"Error details: {e.body}\n")
except Exception as e:
    print("An error occurred: %s\n" % e)

Experiment created successfully: 
None
Experiment title updated successfully:
{'_date': '2024-11-07',
 'access_key': None,
 'body': '<h1><span style="font-size:14pt;">目的:</span></h1>\n'
         '<p>\xa0</p>\n'
         '<h1><strong><span '
         'style="font-size:14pt;">目的組成:<br></span></strong></h1>\n'
         '<table>\n'
         '\n'
         '<tr style="background-color:#c2e0f4;">\n'
         '<td style="border-color:#95a5a6;border-style:solid;">PDP (T/F)</td>\n'
         '<td style="border-color:#95a5a6;border-style:solid;">サンプルID '
         '*1<br>Sample ID</td>\n'
         '<td '
         'style="border-color:#95a5a6;border-style:solid;">目的化合物名<br>Target '
         'Compound</td>\n'
         '<td style="border-color:#95a5a6;border-style:solid;">増仕込み量 *2<br>+ '
         'α</td>\n'
         '<td style="border-color:#95a5a6;border-style:solid;">元素比率<br>Element '
         'Ratio</td>\n'
         '<td style="border-color:#95a5a6;border-style:solid;">合成方法 '
         '*3<br>Method

### 実験ノートリストの確認
作成した全実験ノートを出力する。

In [9]:
# 全実験ノートの確認
try:
    # デフォルト設定で実験ノートを取得
    print("デフォルト設定で実験ノートを取得:")
    experimentsList = experimentsApi.read_experiments()
    print(f"Number of experiments: {len(experimentsList)}")
    
    # 必要なフィールドを抽出してリストに格納
    exp_data = []
    for experiment in experimentsList:
        exp_dict = experiment.to_dict()
        exp_data.append({
            '_date': exp_dict.get('_date'),
            'category_title': exp_dict.get('category_title'),
            'category': exp_dict.get('category'),
            'fullname': exp_dict.get('fullname'),
            'id': exp_dict.get('id'),
            'tags': exp_dict.get('tags'),
            'tags_id': exp_dict.get('tags_id'),
            'up_item_id': exp_dict.get('up_item_id')
        })

    # データフレームに変換
    experiments_df = pd.DataFrame(exp_data)
    experiments_df['_date'] = pd.to_datetime(experiments_df['_date'])   # _date列をdatetime型に変換
    experiments_df = experiments_df.sort_values(by='_date', ascending=False)    # 作成日が新しい順にソート
    print(experiments_df)
    
except ApiException as e:
    print("ExperimentsApiを呼び出す際の例外: %s\n" % e)

デフォルト設定で実験ノートを取得:
Number of experiments: 12
        _date category_title category     fullname    id           tags  \
0  2024-11-07           None     None  jemyung lee  1906  Catalyst|Zinc   
1  2024-11-06           None     None  jemyung lee  1900  Catalyst|Zinc   
2  2024-10-28           None     None  jemyung lee  1848  Catalyst|Zinc   
3  2024-10-28           None     None  jemyung lee  1855  Catalyst|Zinc   
4  2024-10-28           None     None  jemyung lee  1847  Catalyst|Zinc   
5  2024-10-28           None     None  jemyung lee  1845  Catalyst|Zinc   
6  2024-10-28           None     None  jemyung lee  1843  Catalyst|Zinc   
7  2024-10-28           None     None  jemyung lee  1842  Catalyst|Zinc   
8  2024-10-22           None     None  jemyung lee  1796  Catalyst|Zinc   
9  2024-10-22           None     None  jemyung lee  1795  Catalyst|Zinc   
10 2024-10-22           None     None  jemyung lee  1794  Catalyst|Zinc   
11 2024-10-22           None     None  jemyung lee  1793

### 最新の実験ノートを取得
実験ノートリストから最新の実験ノートを取得します。</br>
取得した実験ノートのIDが表示されます。

In [10]:
try:
    experiments_list = experimentsApi.read_experiments()
    print(f"Number of experiments: {len(experiments_list)}")
    
    # 最も新しい実験ノートを取得
    if experiments_list:
        experiments_list = sorted(experiments_list, key=lambda x: x._date, reverse=True)  # 作成日でソート
        experiment_id = experiments_list[0].id  # 最新の実験ノートのIDを取得
        print(f"Selected (the latest) experiment ID: {experiment_id}")        
    else:
        print("No experiments found.")
        exit()

except ApiException as e:
    print(f"Exception when calling ExperimentsApi->read_experiments: {e}\n")
    exit()

Number of experiments: 12
Selected (the latest) experiment ID: 1906


## 実験ノートの内容更新

### 更新内容の入力
実験ノートにアップデートするテーブルデータを入力します。

In [11]:
# 更新する実験ノートの内容を入力
update_contents = []
update_contents.append({
            'PDC': "11",
            'SampleID': "1001",
            'TargetCompound': "55",
            'Alpha': "25",
            'ElementRatio': "2",
            'Method': "150",
            'WC': "30",
            'FWC': "50"
        })
update_contents.append({
            'PDC': "11",
            'Element': "55",
            'IASO': "25",
            'Mw': "2",
            'IMR': "150",
            'IWR': "30",
            'IW': "50",
            'MWR': "123",
            'MW': "456"    
        })	
update_response = ''

### 実験ノートの内容を更新
最新の実験ノートの詳細を取得して実験ノートの内容を更新します。

In [12]:
# 最新の実験ノートの詳細を取得
try:
    latest_experiment_details = experimentsApi.get_experiment(experiment_id)
    current_body = latest_experiment_details.body
    current_title = latest_experiment_details.title

    # BeautifulSoupを使ってHTMLを解析
    soup = BeautifulSoup(current_body, 'html.parser')

    # 例として、特定の表のセルの内容を更新する
    
    tables = soup.find_all('table')
    
    for i, cnts in enumerate(update_contents):
        table = tables[i]
        if table:
            rows = table.find_all('tr')
            if len(rows) > 1:
                items = [x.string for x in rows[1].find_all('td')]
                cells = rows[2].find_all('td')
                if len(cells) >= len(cnts):
                    for c_itm in cnts.keys():
                        ci = items.index(c_itm)
                        cells[ci].string = cnts[c_itm]
                else:
                    raise ValueError(f"Cell(s) ({len(cells)}) is(are) fewer than update contents ({len(cnts)})")
            else:
                raise ValueError(f"Insufficient rows ({len(rows)})")
        else:
            raise ValueError(f"Table(s) ({len(soup.findAll('tr'))}) is(are) fewer than update contents ({len(update_contents)})")

    # 更新されたHTMLを文字列に戻す
    updated_body = str(soup)

    # 新しい内容をもとに既存の内容を更新
    update_body = {
        "body": updated_body,  # 更新されたHTML
    }

    # 実験ノートの内容を更新
    update_response = experimentsApi.patch_experiment(experiment_id, body=update_body)
    print("Experiment has updated successfully.")
    
except ApiException as e:
    print(f"Exception when calling ExperimentsApi->patch_experiment: {e}\n")
    if e.body:
        print(f"Error details: {e.body}\n")
except Exception as e:
    print(f"An error occurred: {e}\n")

Experiment has updated successfully.


### 更新Raw requestの確認
下記のコードを実行すると、APIを通じて配信したリクエストを確認することができます。


In [13]:
try:
    # リクエスト内容を表示
    print("Request body:")
    print(json.dumps(update_body, ensure_ascii=False, indent=2))
    
    if update_response:
        print(update_response)
    
except Exception as e:
    print(f"An error occurred: {e}\n")

Request body:
{
  "body": "<h1><span style=\"font-size:14pt;\">目的:</span></h1>\n<p> </p>\n<h1><strong><span style=\"font-size:14pt;\">目的組成:<br/></span></strong></h1>\n<table>\n<tr style=\"background-color:#c2e0f4;\">\n<td style=\"border-color:#95a5a6;border-style:solid;\">PDP (T/F)</td>\n<td style=\"border-color:#95a5a6;border-style:solid;\">サンプルID *1<br/>Sample ID</td>\n<td style=\"border-color:#95a5a6;border-style:solid;\">目的化合物名<br/>Target Compound</td>\n<td style=\"border-color:#95a5a6;border-style:solid;\">増仕込み量 *2<br/>+ α</td>\n<td style=\"border-color:#95a5a6;border-style:solid;\">元素比率<br/>Element Ratio</td>\n<td style=\"border-color:#95a5a6;border-style:solid;\">合成方法 *3<br/>Method</td>\n<td style=\"border-color:#95a5a6;border-style:solid;\">生成物量 [g]<br/>Weight of Compound</td>\n<td style=\"border-color:#95a5a6;border-style:solid;\">生成物分子量 [g/mol]<br/>Formura Weight of Compound</td>\n</tr>\n<tr style=\"background-color:#c2e0f4;\">\n<td style=\"border-color:#95a5a6;border-style:s

#### 次へ 
お疲れ様でした。次のステップのための準備ができました。次のステップに進むには、ここをクリックしてください。